In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
import re
import requests
import os

from sklearn.model_selection import TimeSeriesSplit, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer, RobustScaler, FunctionTransformer
from sklearn.linear_model import LassoCV, Ridge, Lasso
from sklearn.feature_selection import f_regression, RFE, SelectKBest, SequentialFeatureSelector, RFECV
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import r2_score, mean_absolute_error

In [3]:
df_train = pd.read_csv(os.path.join("..", "data", "dataset_preprocesamiento_train.csv"))
df_test = pd.read_csv(os.path.join("..", "data", "dataset_preprocesamiento_test.csv"))

## Fase 2

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
import re
import requests

print(f"Train inicial: {df_train.shape} | Test inicial: {df_test.shape}")


# --- 1. "APRENDER" (FIT) SOLO DE DF_TRAIN ---
# (El único paso con Data Leakage potencial, ahora es seguro)
print("Aprendiendo 'top5' features solo de df_train...")
ycol = 'Frio' # Usamos el 'Frio' de hoy como base para correlacionar

num_train = df_train.select_dtypes(include=[np.number]).copy()
candidatas_train = [c for c in num_train.columns if c != ycol and c != 'y' and num_train[c].notna().any()]

top5_features_aprendidas = (
    num_train[candidatas_train].corrwith(num_train[ycol])
    .abs()
    .sort_values(ascending=False)
    .head(5)
    .index.tolist()
)
print(f"Top 5 features aprendidas: {top5_features_aprendidas}")


# --- 2. JUNTAR Y TRANSFORMAR (Aplicar Feature Engineering) ---

# 2a. Juntamos los dataframes para que los lags/rolling fluyan
split_date = df_test['dia'].min() # Guardamos la fecha de corte
df_full = pd.concat([df_train, df_test]).sort_values(by='dia').reset_index(drop=True)

print(f"DataFrame unificado para procesar: {df_full.shape}")

# 2b. APLICAMOS TODO EL FEATURE ENGINEERING AL DATAFRAME JUNTO
# (Tu código, pero aplicado a 'df_full' y con los cambios que pediste)

# --- Lags y Rolling (Tus cambios) ---
ycol = 'Frio'
df_full[f'{ycol}_lag7'] = df_full[ycol].shift(7)

y_obs = df_full[ycol].shift(1) # Frio de ayer
df_full[f'{ycol}_ma3'] = y_obs.rolling(window=3, min_periods=1).mean()
df_full[f'{ycol}_ma7'] = y_obs.rolling(window=7, min_periods=1).mean()

# --- Fechas Cíclicas y Finde ---
df_full['fecha'] = pd.to_datetime(df_full['dia'], format='%Y-%m-%d', errors='coerce')
dow = df_full['fecha'].dt.dayofweek
df_full['dow_sin'] = np.sin(2*np.pi * dow / 7)
df_full['dow_cos'] = np.cos(2*np.pi * dow / 7)
m  = df_full['fecha'].dt.month
m0 = (m - 1)
df_full['mes_sin'] = np.sin(2*np.pi * m0 / 12)
df_full['mes_cos'] = np.cos(2*np.pi * m0 / 12)
df_full['fin_de_semana'] = df_full['fecha'].dt.dayofweek.isin([5, 6]).astype(int)

# --- Ratios e Interacciones (usando la 'top5_features_aprendidas') ---
top5 = top5_features_aprendidas # Usamos la variable segura
for a, b in combinations(top5, 2):
    df_full[f'{a}x{b}'] = df_full[a] * df_full[b]
for cyc in ['dow_sin', 'dow_cos', 'mes_sin', 'mes_cos']:
    if cyc in df_full.columns:
        for c in top5:
            df_full[f'{c}_x_{cyc}'] = df_full[c] * df_full[cyc]
if 'fin_de_semana' in df_full.columns:
    for c in top5:
        df_full[f'{c}_x_finde'] = df_full[c] * df_full['fin_de_semana']

# --- Ratios de Áreas ---
AREAS = {
    "Elaboración": r"Elab|Elabor|Coci|Cocina|Mosto|Lauter|Macer|Paste",
    "Envasado":    r"Envas|Llen|Linea|L[2345]\b",
    "Bodega":      r"Bodega|Bodeg",
    "Servicios":   r"Servicios|Vapor|Gas|Agua|Aire|Caldera|Compres|Chiller|Sala",
    "Sala_Maq":    r"Sala.*Maq",
}
def safe_div(a, b):
    # Usamos np.where para evitar dividir por cero
    return np.where(b != 0, a / b, 0)

num_cols = df_full.select_dtypes(include=[np.number]).columns
area_cols = {}
for area, pat in AREAS.items():
    regex = re.compile(pat, flags=re.IGNORECASE)
    cols = [c for c in num_cols if regex.search(c)]
    area_cols[area] = cols

area_sum = {}
for area, cols in area_cols.items():
    if cols:
        df_full[f'{area}_sum'] = df_full[cols].sum(axis=1, skipna=True)
        area_sum[area] = df_full[f'{area}_sum']
    else:
        df_full[f'{area}_sum'] = 0.0
        area_sum[area] = df_full[f'{area}_sum']

areas_presentes = list(area_sum.keys())
total_sel = sum(area_sum[a] for a in areas_presentes)
df_full['Consumo_Total_Areas'] = total_sel

for a in areas_presentes:
    df_full[f'{a}_share'] = safe_div(df_full[f'{a}_sum'], df_full['Consumo_Total_Areas'])
for a, b in combinations(areas_presentes, 2):
    df_full[f'ratio_{a}_sobre_{b}'] = safe_div(df_full[f'{a}_sum'], df_full[f'{b}_sum'])
    df_full[f'ratio_{b}_sobre_{a}'] = safe_div(df_full[f'{b}_sum'], df_full[f'{a}_sum'])

df_full = df_full.replace([np.inf, -np.inf], np.nan)

# --- Estacionalidad y Clima ---
m = df_full['fecha'].dt.month
est_idx = np.select(
    [m.isin([12, 1, 2]), m.isin([3, 4, 5]), m.isin([6, 7, 8]), m.isin([9, 10, 11])],
    [0, 1, 2, 3], default=np.nan
)
df_full['estacion_sin'] = np.sin(2*np.pi * est_idx / 4)
df_full['estacion_cos'] = np.cos(2*np.pi * est_idx / 4)

start_date = df_full["fecha"].min().date().isoformat()
end_date   = df_full["fecha"].max().date().isoformat()
print(f"Buscando clima de {start_date} a {end_date}...")
lat, lon = 32.56717, -116.62509
url = (
    "https://archive-api.open-meteo.com/v1/archive"
    f"?latitude={lat}&longitude={lon}"
    f"&start_date={start_date}&end_date={end_date}"
    "&daily=temperature_2m_mean"
    "&timezone=auto"
)
data = requests.get(url, timeout=60).json()
wx_d = pd.DataFrame({
    "fecha": pd.to_datetime(data["daily"]["time"]),
    "t2m_mean_C": data["daily"]["temperature_2m_mean"],
})
df_full = df_full.merge(wx_d, on="fecha", how="left")


# --- 3. LIMPIAR Y SEPARAR ---

# 3a. Borrar las primeras 7 filas (como dijo tu profe)
# Usamos .iloc[7:] para quedarnos con todo DESPUÉS de la fila 7
df_processed = df_full.iloc[7:].copy()
print(f"Filas eliminadas por NaNs de lags: 7. Quedan {df_processed.shape[0]} filas.")

# 3b. Rellenar cualquier otro NaN que quede (del merge de clima, ratios, etc.)
df_processed = df_processed.fillna(0) # O puedes usar un imputador aquí si prefieres

# 3c. Volver a separar
df_train_final = df_processed[df_processed['dia'] < split_date].copy()
df_test_final = df_processed[df_processed['dia'] >= split_date].copy()


print("\n--- ¡PROCESO COMPLETADO SIN DATA LEAKAGE! ---")
print(f"Shape Train final: {df_train_final.shape}")
print(f"Shape Test final:  {df_test_final.shape}")

Train inicial: (913, 118) | Test inicial: (297, 118)
Aprendiendo 'top5' features solo de df_train...
Top 5 features aprendidas: ['Sala Maq (Kw)__Consolidado EE', 'Servicios (Kw)__Consolidado EE', 'KW Gral Planta__Consolidado EE', 'Planta (Kw)__Consolidado EE', 'Agua Planta (Hl)__Consolidado Agua']
DataFrame unificado para procesar: (1210, 118)
Buscando clima de 2020-07-01 a 2023-10-25...
Filas eliminadas por NaNs de lags: 7. Quedan 1203 filas.

--- ¡PROCESO COMPLETADO SIN DATA LEAKAGE! ---
Shape Train final: (906, 196)
Shape Test final:  (297, 196)


### Selección de variables

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression, mutual_info_regression, SequentialFeatureSelector, SelectFromModel
from sklearn.linear_model import LassoCV, ElasticNetCV, Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import matplotlib.pyplot as plt

# (Asumo que df_train_final y df_test_final ya existen)

# Definimos la columna objetivo (target)
ycol = 'y'

# --- Creamos X_train, y_train, X_test, y_test ---
# (Esta vez, X_train y X_test se quedan con las columnas de texto por ahora)
X_train_full = df_train_final.drop(columns=[ycol], errors='ignore')
y_train = df_train_final[ycol]

X_test_full = df_test_final.drop(columns=[ycol], errors='ignore')
y_test = df_test_final[ycol]

# --- ¡ESTE ES EL PASO CLAVE QUE FALTABA! ---
# 1. Creamos una lista de features que SÍ son numéricas
numeric_features = X_train_full.select_dtypes(include=np.number).columns.tolist()

# 2. Creamos los DataFrames X_train y X_test solo con esas columnas
X_train = X_train_full[numeric_features].copy()
X_test = X_test_full[numeric_features].copy()
# (Nos aseguramos de que X_test tenga las mismas columnas en el mismo orden)
X_test = X_test[X_train.columns]

print(f"Listos para seleccionar. {len(numeric_features)} features numéricas iniciales.")

Listos para seleccionar. 192 features numéricas iniciales.


In [6]:
# --- A. Filtro por Varianza (VarianceThreshold) ---
# Primero escalamos, porque la varianza es sensible a la escala
scaler_var = MinMaxScaler()

# 1. APRENDER (fit) el scaler SOLO en X_train
scaler_var.fit(X_train) 

# 2. APLICAR (transform) en ambos
X_train_scaled_var = pd.DataFrame(scaler_var.transform(X_train), columns=X_train.columns)
X_test_scaled_var = pd.DataFrame(scaler_var.transform(X_test), columns=X_test.columns)

# 3. APRENDER (fit) el selector de varianza SOLO en X_train_scaled
# (Tu código buscaba k=16, pero ahora pedís k=30)
# Vamos a usar SelectKBest con la varianza para pedir k=30
var_selector = SelectKBest(score_func=lambda X, y: X.var(axis=0), k=30)
var_selector.fit(X_train_scaled_var, y_train) # 'y_train' no se usa, pero la API lo pide

# 4. OBTENER las 30 mejores features
features_variance = X_train.columns[var_selector.get_support()]
print(f"\n--- 1. Método Varianza (k=30) --- \n{features_variance.tolist()}")


# --- B. Filtro por F-Score (f_regression) ---
# (Este método necesita que los datos escalados no tengan NaNs)
X_train_scaled_var = X_train_scaled_var.fillna(0)
X_test_scaled_var = X_test_scaled_var.fillna(0)

# 1. APRENDER (fit) el selector SOLO en X_train
f_selector = SelectKBest(score_func=f_regression, k=30)
f_selector.fit(X_train_scaled_var, y_train)

# 2. OBTENER las 30 mejores features
features_fscore = X_train.columns[f_selector.get_support()]
print(f"\n--- 2. Método F-Score (k=30) --- \n{features_fscore.tolist()}")


# --- C. Filtro por Información Mutua (mutual_info_regression) ---
# 1. APRENDER (fit) el selector SOLO en X_train
mi_selector = SelectKBest(score_func=mutual_info_regression, k=30)
mi_selector.fit(X_train_scaled_var, y_train)

# 2. OBTENER las 30 mejores features
features_mi = X_train.columns[mi_selector.get_support()]
print(f"\n--- 3. Método Mutual Info (k=30) --- \n{features_mi.tolist()}")




--- 1. Método Varianza (k=30) --- 
['Unnamed: 0', 'Linea 3 (Kw)__Consolidado EE', 'Linea 2 (Kw)__Consolidado EE', 'Vapor L3__Consolidado GasVapor', 'Totalizador_Aire_L3__Consolidado Aire', 'Totalizador_Aire_L5__Consolidado Aire', 'dow_sin', 'dow_cos', 'mes_sin', 'mes_cos', 'fin_de_semana', 'KW Gral Planta__Consolidado EE_x_dow_sin', 'Planta (Kw)__Consolidado EE_x_dow_sin', 'Servicios (Kw)__Consolidado EE_x_dow_cos', 'KW Gral Planta__Consolidado EE_x_dow_cos', 'Planta (Kw)__Consolidado EE_x_dow_cos', 'Agua Planta (Hl)__Consolidado Agua_x_dow_cos', 'Servicios (Kw)__Consolidado EE_x_mes_sin', 'KW Gral Planta__Consolidado EE_x_mes_sin', 'Planta (Kw)__Consolidado EE_x_mes_sin', 'Agua Planta (Hl)__Consolidado Agua_x_mes_sin', 'KW Gral Planta__Consolidado EE_x_mes_cos', 'Planta (Kw)__Consolidado EE_x_mes_cos', 'Sala Maq (Kw)__Consolidado EE_x_finde', 'Servicios (Kw)__Consolidado EE_x_finde', 'KW Gral Planta__Consolidado EE_x_finde', 'Planta (Kw)__Consolidado EE_x_finde', 'Agua Planta (Hl)__C

In [7]:
# --- D. Método Embebido (LassoCV) ---
# Para Lasso/ElasticNet, es mejor usar StandardScaler (media 0, var 1)
scaler_reg = StandardScaler()

# 1. APRENDER (fit) el scaler SOLO en X_train
scaler_reg.fit(X_train)

# 2. APLICAR (transform) en ambos
X_train_scaled_reg = pd.DataFrame(scaler_reg.transform(X_train.fillna(0)), columns=X_train.columns)
X_test_scaled_reg = pd.DataFrame(scaler_reg.transform(X_test.fillna(0)), columns=X_test.columns)

# 3. APRENDER (fit) el modelo Lasso SOLO en X_train
lasso = LassoCV(cv=5, random_state=42).fit(X_train_scaled_reg, y_train)

# 4. OBTENER las features (Lasso elige cuántas, no podemos forzar k=30)
#    Así que tomaremos las 30 con coeficientes más altos (en valor absoluto)
coef_lasso = pd.Series(lasso.coef_, index=X_train.columns)
features_lasso = coef_lasso.abs().sort_values(ascending=False).head(30).index.tolist()
print(f"\n--- 4. Método Lasso (top 30) --- \n{features_lasso}")


# --- E. Método Embebido (ElasticNetCV) ---
# 1. APRENDER (fit) el modelo ElasticNet SOLO en X_train
elastic = ElasticNetCV(cv=5, random_state=42).fit(X_train_scaled_reg, y_train)

# 2. OBTENER las 30 mejores features (mismo método que Lasso)
coef_elastic = pd.Series(elastic.coef_, index=X_train.columns)
features_elastic = coef_elastic.abs().sort_values(ascending=False).head(30).index.tolist()
print(f"\n--- 5. Método ElasticNet (top 30) --- \n{features_elastic}")

c:\Users\angim\miniconda3\envs\cervecera_env\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.957e+06, tolerance: 4.830e+06
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\angim\miniconda3\envs\cervecera_env\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.414e+06, tolerance: 4.830e+06
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\angim\miniconda3\envs\cervecera_env\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features o


--- 4. Método Lasso (top 30) --- 
['Frio', 'Frio_ma7', 'mes_cos', 'ratio_Bodega_sobre_Sala_Maq', 'ratio_Servicios_sobre_Bodega', 'mes_sin', 'estacion_cos', 'Red Paste L4__Consolidado Agua', 'ET Servicios / Hl__Consolidado KPI', 'Sala Maq (Kw)__Consolidado EE_x_dow_cos', 'Aire Envasado (M3)__Consolidado Aire', 'Agua Bodega / Hl__Consolidado KPI', 'KW Gral Planta__Consolidado EE', 'Aire Bodega / Hl__Consolidado KPI', 'Sala Maq (Kw)__Consolidado EE', 'Frio_lag7', 'Servicios (Kw)__Consolidado EE', 'Agua Linea 5/Hl__Consolidado KPI', 'ET Linea 3/Hl__Consolidado KPI', 'Agua Linea 4/Hl__Consolidado KPI', 'Agua Linea 2/Hl__Consolidado KPI', 'EE Cocina / Hl__Consolidado KPI', 'Unnamed: 0', 'ET Elab/Hl__Consolidado KPI', 'EE Linea 4 / Hl__Consolidado KPI', 'ET Bodega/Hl__Consolidado KPI', 'EE Caldera / Hl__Consolidado KPI', 'Aire L4 / Hl__Consolidado KPI', 'Agua Cocina / Hl__Consolidado KPI', 'EE Linea 2 / Hl__Consolidado KPI']

--- 5. Método ElasticNet (top 30) --- 
['Frio_ma7', 'Frio_ma3', 'm

In [8]:
# --- F. Método Wrapper (SequentialFeatureSelector) ---
# (Usamos los datos escalados con StandardScaler, que ya están limpios de NaNs)
print("\n--- 6. Método Wrapper (SFS) ---")
print("Iniciando SFS... (Esto puede tardar varios minutos)...")

# 1. Definir el modelo que usará SFS para evaluar
# (Tu código usaba GradientBoosting, es una buena elección)
modelo_sfs = GradientBoostingRegressor(random_state=42)

# 2. APRENDER (fit) el selector SFS SOLO en X_train
sfs = SequentialFeatureSelector(modelo_sfs, 
                              n_features_to_select=30, # ¡Aquí pones 30!
                              direction='forward', # 'forward' es más rápido que 'backward'
                              cv=3, # 3 splits es más rápido que 5
                              n_jobs=-1) # Usar todos los cores

sfs.fit(X_train_scaled_reg, y_train)

# 3. OBTENER las 30 mejores features
features_sfs = X_train.columns[sfs.get_support()]
print(f"SFS completado.\nFeatures seleccionadas: \n{features_sfs.tolist()}")


--- 6. Método Wrapper (SFS) ---
Iniciando SFS... (Esto puede tardar varios minutos)...


KeyboardInterrupt: 


--- 6. Método Wrapper (SFS) ---
Iniciando SFS... (Esto puede tardar varios minutos)...
SFS completado.
Features seleccionadas: 
['Agua Elab / Hl__Consolidado KPI', 'Agua Planta de Agua/Hl__Consolidado KPI', 'Produccion Agua / Hl__Consolidado KPI', 'EE Envasado / Hl__Consolidado KPI', 'Agua Bodega / Hl__Consolidado KPI', 'EE Elaboracion / Hl__Consolidado KPI', 'EE Linea 4 / Hl__Consolidado KPI', 'Aire Elaboracion / Hl__Consolidado KPI', 'KW Enfluentes Hidr__Totalizadores Energia', 'KW Obrador Contratistas__Totalizadores Energia', 'KW Mycom 3__Totalizadores Energia', 'KW Mycom 7__Totalizadores Energia', 'FC Lavadora L3__Consolidado Agua', 'Produccion (Hl)__Consolidado Agua', 'Frio', 'Frio_lag7', 'Frio_ma7', 'dow_sin', 'dow_cos', 'mes_sin', 'fin_de_semana', 'Servicios (Kw)__Consolidado EE_x_dow_sin', 'KW Gral Planta__Consolidado EE_x_dow_sin', 'KW Gral Planta__Totalizadores Energia_x_dow_sin', 'Planta (Kw)__Consolidado EE_x_dow_sin', 'KW Gral Planta__Consolidado EE_x_dow_cos', 'KW Gral Planta__Totalizadores Energia_x_dow_cos', 'Servicios (Kw)__Consolidado EE_x_finde', 'Bodega_sum', 'estacion_sin']

In [13]:
rf = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

perm_train = permutation_importance(
    rf, X_train, y_train,
    n_repeats=5, random_state=42, n_jobs=-1,
    scoring="neg_mean_absolute_error"
)

In [14]:
perm_imp_train = pd.Series(perm_train.importances_mean, index=X_train.columns).sort_values(ascending=False)

features_rf = perm_imp_train.head(30).index.tolist()

# Subsets coherentes en train/test
X_train_sel = X_train[features_rf].copy()
X_test_sel  = X_test[features_rf].copy()


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

rf_sel = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
rf_sel.fit(X_train_sel, y_train)

y_pred = rf_sel.predict(X_test_sel)

In [15]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np

# --- 1. Definir el Modelo y los Sets de Features ---

# El modelo que querés usar para la evaluación final
model = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)

# Un diccionario con todas tus listas de 30 features (de los pasos anteriores)
listas_de_features = {
    "Varianza": features_variance,
    "F-Score": features_fscore,
    "Mutual_Info": features_mi,
    "Lasso": features_lasso,
    "ElasticNet": features_elastic,
    "Wrapper_SFS": features_sfs,
    "Random Forest": features_rf
}

# (Tus dataframes X_train, y_train, X_test, y_test ya existen)
print("--- INICIANDO EVALUACIÓN FINAL EN SET DE TEST ---")


# --- 2. La Función de Evaluación (simple, como la tuya) ---

def evaluar_en_test(X_train_subset, y_train, X_test_subset, y_test, nombre_metodo):
    """
    Entrena en Train, evalúa en Test y printea las métricas.
    """
    
    # --- A. Escalar (SIN DATA LEAKAGE) ---
    # Limpiamos NaNs que puedan quedar
    X_train_clean = X_train_subset.fillna(0)
    X_test_clean = X_test_subset.fillna(0)
    
    # 1. Aprender (fit) el scaler SOLO en Train
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_clean)
    
    # 2. Aplicar (transform) el scaler en Train y Test
    X_test_scaled = scaler.transform(X_test_clean) # ¡Solo transform!
    
    
    # --- B. Entrenar y Predecir ---
    # 3. Entrenar el modelo
    model.fit(X_train_scaled, y_train)
    
    # 4. Predecir en Test
    y_pred = model.predict(X_test_scaled)
    
    
    # --- C. Calcular Métricas ---
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred) # El MAE que te pide el TP
    
    # 5. Imprimir (como en tu código)
    print(f"\n>>> {nombre_metodo} ({X_train_subset.shape[1]} features) <<<")
    print(f"  R2 (en Test):   {r2:.4f}")
    #print(f"  MSE (en Test):  {mse:.2f}")
    print(f"  MAE (en Test):  {mae:.2f}") # ¡Métrica clave del TP!

    
# --- 3. Loop de Evaluación ---
# (Esto reemplaza tu creación manual de X_l1, X_elastic, etc.)

for nombre, lista_de_features in listas_de_features.items():
    # Seleccionamos las 30 columnas para este método
    X_train_subset = X_train[lista_de_features]
    X_test_subset = X_test[lista_de_features]
    
    # Evaluamos
    evaluar_en_test(X_train_subset, y_train, X_test_subset, y_test, nombre)
    

NameError: name 'features_sfs' is not defined

In [16]:
cols_ok = [c for c in features_rf if c in df_train_final.columns]

# Avisar si alguna feature no aparece en df_train
faltan = sorted(set(features_rf) - set(cols_ok))
if faltan:
    print("No encontré estas columnas en df_train:", faltan)

# Sobrescribe con solo esas columnas
# df_train = df_train_final.loc[:, cols_ok + ['y']].copy()
df_test = df_test_final.loc[:, cols_ok + ['y']].copy()

In [17]:
cols_ok

['Frio_ma7',
 'Frio',
 'ratio_Bodega_sobre_Sala_Maq',
 'ratio_Sala_Maq_sobre_Bodega',
 'Sala Maq (Kw)__Consolidado EExServicios (Kw)__Consolidado EE',
 'Sala Maq (Kw)__Consolidado EE',
 'Bodega_share',
 'Sala Maq (Kw)__Consolidado EE_x_mes_cos',
 'ratio_Servicios_sobre_Bodega',
 'Servicios (Kw)__Consolidado EE_x_mes_cos',
 'ratio_Bodega_sobre_Servicios',
 'Sala_Maq_sum',
 'Sala Maq (Kw)__Consolidado EExPlanta (Kw)__Consolidado EE',
 'Frio_lag7',
 'Frio_ma3',
 'Agua Planta (Hl)__Consolidado Agua_x_mes_cos',
 'Sala Maq (Kw)__Consolidado EE_x_mes_sin',
 'Sala Maq (Kw)__Consolidado EExKW Gral Planta__Consolidado EE',
 'Aire L4 / Hl__Consolidado KPI',
 'KW Gral Planta__Consolidado EE_x_mes_cos',
 'Linea 3 (Kw)__Consolidado EE',
 'Servicios (Kw)__Consolidado EE',
 'Red L1 y L2__Consolidado Agua',
 'Sala Maq (Kw)__Consolidado EExAgua Planta (Hl)__Consolidado Agua',
 'Agua Linea 3/Hl__Consolidado KPI',
 'Sala Maq (Kw)__Consolidado EE_x_dow_cos',
 'CO 2 Linea 4 / Hl__Consolidado KPI',
 'Red Pas

In [ ]:
# --- Tus funciones de transformación (BoxCox, Log, Sqrt) ---
# (Las copio tal cual, están perfectas)
def boxcox_transform(X):
    X = np.where(X <= 0, X + 1e-9, X)
    return PowerTransformer(method='box-cox', standardize=True).fit_transform(X)
def log_transform(X):
    return np.log1p(np.clip(X, a_min=0, a_max=None))
def sqrt_transform(X):
    return np.sqrt(np.abs(X))


# --- 1. PREPARACIÓN DE DATOS (El cambio clave) ---
# (Asumo que df_train y df_test ya existen y están limpios)

ycol = 'y' # Tu columna objetivo (Frio shifteado)
# Columnas que no son features (categóricas o de info)
non_features = [ycol, 'dia'] 

# 1a. Crear X_train, y_train (para ENTRENAR el GridSearchCV)
X_train_full = df_train.drop(columns=non_features, errors='ignore')
y_train = df_train[ycol]

# 1b. Crear X_test, y_test (para EVALUAR AL FINAL)
X_test_full = df_test.drop(columns=non_features, errors='ignore')
y_test = df_test[ycol]

# 1c. Detectar features numéricas (APRENDIENDO SOLO DE X_TRAIN)
numeric_features = X_train_full.select_dtypes(include=np.number).columns.tolist()

# 1d. Asegurarnos que X_train y X_test tengan las mismas columnas numéricas
X_train = X_train_full[numeric_features].fillna(0)
X_test = X_test_full[numeric_features].fillna(0) # Rellenamos NaNs simple

print(f"Listos. Usando {len(numeric_features)} features numéricas.")
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")


# --- 2. DEFINICIÓN DEL PIPELINE (Tu código, sin cambios) ---
# (El imputer SimpleImputer(median) es bueno para velocidad en el gridsearch)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', 'passthrough'),
    ('power', 'passthrough')
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge()) # Ponemos un modelo default, se va a reemplazar
])


# --- 3. ESPACIO DE BÚSQUEDA (Tu código, sin cambios) ---
param_grid = {
    # ¡DOBLE GUION BAJO!
    'preprocessor__num__scaler': [ 
        'passthrough',
        StandardScaler(),
        MinMaxScaler(),
        RobustScaler()
    ],
    
    # ¡DOBLE GUION BAJO!
    'preprocessor__num__power': [ 
        PowerTransformer(method='yeo-johnson'),
        FunctionTransformer(boxcox_transform, validate=False),
        FunctionTransformer(log_transform, validate=False),
        FunctionTransformer(sqrt_transform, validate=False),
        'passthrough'
    ],
    
    # 'regressor' está bien con uno solo porque es un paso directo
    # del 'model_pipeline'
    'regressor': [
        XGBRegressor(
            n_estimators=600,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.0,
            reg_lambda=1.0,
            random_state=42,
            n_jobs=-1,
            tree_method="hist"
        ),
        RandomForestRegressor(
            n_estimators=600,
            max_depth=None,
            min_samples_split=2,
            min_samples_leaf=1,
            max_features="sqrt",
            bootstrap=True,
            n_jobs=-1,
            random_state=42
        ),
        LGBMRegressor(
            n_estimators=1000,
            learning_rate=0.05,
            num_leaves=31,
            max_depth=-1,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_lambda=1.0,
            random_state=42,
            n_jobs=-1
        ),
        Ridge(
            alpha=1.0,
            fit_intercept=True
        ),
        Lasso(
            alpha=0.001,
            max_iter=10000,
            fit_intercept=True
        )
    ],
}


# --- 4. GridSearchCV (EL FIT SE HACE SOLO EN TRAIN) ---
# (Usamos TimeSeriesSplit para Cross-Validation, es mejor para series de tiempo)
# tscv = TimeSeriesSplit(n_splits=5) # Opcional, pero recomendado
cv = KFold(n_splits=5, shuffle=True, random_state=42) # Tu CV original

grid_search = GridSearchCV(
    model_pipeline, 
    param_grid, 
    cv=cv, # Reemplaza por tscv si quieres probar
    scoring='neg_mean_absolute_error', # O 'neg_mean_absolute_error' para el MAE
    n_jobs=-1, 
    verbose=1
)

print("Iniciando la búsqueda (GridSearch) SOLO EN DF_TRAIN...")

# ¡AQUÍ ESTÁ LA CLAVE!
# Entrenamos el buscador de modelos SOLO con los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros (encontrados en Train)
print("\nLos mejores parámetros encontrados (en train) son:", grid_search.best_params_)
print("El mejor MAE (en CV de train) es: {:.4f}".format(grid_search.best_score_))

# Guardamos el MEJOR pipeline encontrado
best_pipeline = grid_search.best_estimator_


# --- 5. EVALUACIÓN FINAL (SOLO EN TEST) ---
print("\n--- Evaluación Final en DF_TEST (Datos nunca vistos) ---")

# Usamos el mejor pipeline para predecir en X_test
y_pred = best_pipeline.predict(X_test)

# Calcular métricas en Test
r2_test = r2_score(y_test, y_pred)
mae_test = mean_absolute_error(y_test, y_pred) # La métrica que te pide el TP
rmse_test = np.sqrt(mean_absolute_error(y_test, y_pred))

print(f"R² (en Test):  {r2_test:.4f}")
print(f"MAE (en Test): {mae_test:.2f}")
print(f"RMSE (en Test): {rmse_test:.2f}")



Listos. Usando 30 features numéricas.
X_train shape: (906, 30), X_test shape: (297, 30)
Iniciando la búsqueda (GridSearch) SOLO EN DF_TRAIN...
Fitting 5 folds for each of 100 candidates, totalling 500 fits


c:\Users\out6let\miniconda3\envs\cervecera_env\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
75 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
75 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\out6let\miniconda3\envs\cervecera_env\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\out6let\miniconda3\envs\cervecera_env\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\out6let\miniconda3\envs\cervecera_e


Los mejores parámetros encontrados (en train) son: {'preprocessor__num__power': PowerTransformer(), 'preprocessor__num__scaler': MinMaxScaler(), 'regressor': Lasso(alpha=0.001, max_iter=10000)}
El mejor MAE (en CV de train) es: -2813.3932

--- Evaluación Final en DF_TEST (Datos nunca vistos) ---
R² (en Test):  0.6114
MAE (en Test): 2685.65
RMSE (en Test): 51.82


c:\Users\out6let\miniconda3\envs\cervecera_env\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.653e+09, tolerance: 6.005e+06
  model = cd_fast.enet_coordinate_descent(


In [19]:
import datetime
import json

In [22]:
# --- 1. Definir el pipeline de preprocesamiento ganador ---
preproc_pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('power', PowerTransformer(method='yeo-johnson')) 
])

# --- 2. Asumir que X_train, y_train, df_train, df_test existen ---
# X_train: SOLO las 30 features numéricas de train
# y_train: La serie 'y' de train
# df_train: El dataframe COMPLETO de train (con 'dia', 'y', 'Frio', etc.)
# df_test: El dataframe COMPLETO de test

print(f"Forma de X_train (numérico): {X_train.shape}")
print(f"Forma de df_train (completo): {df_train.shape}")


# --- 3. Aplicar (FIT y TRANSFORM) en X_train (SIN LEAKAGE) ---
print("\nAjustando (fit) el pipeline ganador en X_train (numérico)...")
preproc_pipeline.fit(X_train)

# Transformamos X_train y X_test
X_train_escalado_array = preproc_pipeline.transform(X_train)
X_test_escalado_array = preproc_pipeline.transform(X_test)


# --- 4. CONSTRUIR LOS DATAFRAMES FINALES (EL PASO CLAVE) ---

# --- df_train_escalado ---
# 4a. Empezar con una copia del df_train original (que tiene 'dia', 'y', 'Frio', etc.)
df_train_escalado_final = df_train.copy()

# 4b. Crear un DataFrame con los datos escalados y los nombres de columna correctos
df_train_scaled_features = pd.DataFrame(
    X_train_escalado_array, 
    columns=X_train.columns, 
    index=X_train.index
)

# 4c. Actualizar las 30 columnas numéricas en la copia final con los valores escalados
# (.update() reemplaza los valores de las columnas en un df con los valores de otro)
df_train_escalado_final.update(df_train_scaled_features)


# --- df_test_escalado ---
# 4a. Empezar con una copia del df_test original
df_test_escalado_final = df_test.copy()

# 4b. Crear un DataFrame con los datos escalados
df_test_scaled_features = pd.DataFrame(
    X_test_escalado_array, 
    columns=X_test.columns, 
    index=X_test.index
)

# 4c. Actualizar las 30 columnas numéricas en la copia final
df_test_escalado_final.update(df_test_scaled_features)


# --- 5. Guardar en CSV ---
ruta_train_csv = '../data/dataset_train_escalado.csv'
ruta_test_csv = '../data/dataset_test_escalado.csv'

df_train_escalado_final.to_csv(ruta_train_csv, index=False)
df_test_escalado_final.to_csv(ruta_test_csv, index=False)

info_linaje = {
    'descripcion': 'Dataset final procesado, con lags, features ciclicas y ratios.',
    'fuente_original': 'data/dataset_train_escalado.csv y data/dataset_test_escalado.csv',
    'script_transformacion': 'notebooks/preprocesamiento.ipynb', # O el nombre de tu script
    'fecha_creacion': datetime.datetime.now().isoformat()
    # (Buena práctica: añadir el hash del commit de Git)
    # 'git_commit_hash': '...' 
}

# 2b. Guardás el "recibo" en formato JSON
ruta_json = '../data/data_lineage.json'
with open(ruta_json, 'w') as f:
    json.dump(info_linaje, f, indent=4)

print(f"¡Linaje de datos guardado en {ruta_json}!")

print(f"\n¡Listo! Archivos COMPLETOS guardados:")
print(f"Train escalado: {ruta_train_csv}")
print(f"Test escalado:  {ruta_test_csv}")


Forma de X_train (numérico): (906, 192)
Forma de df_train (completo): (913, 118)

Ajustando (fit) el pipeline ganador en X_train (numérico)...


C:\Users\angim\AppData\Local\Temp\ipykernel_8424\3049990754.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.00000000e+00  1.00000000e+00  2.00000000e+00  3.00000000e+00
  4.00000000e+00  5.00000000e+00  6.00000000e+00 -1.78358944e+00
 -1.77937295e+00 -1.77515763e+00 -1.77094348e+00 -1.76673049e+00
 -1.76251866e+00 -1.75830800e+00 -1.75409850e+00 -1.74989016e+00
 -1.74568297e+00 -1.74147694e+00 -1.73727206e+00 -1.73306834e+00
 -1.72886576e+00 -1.72466434e+00 -1.72046406e+00 -1.71626494e+00
 -1.71206695e+00 -1.70787011e+00 -1.70367441e+00 -1.69947985e+00
 -1.69528643e+00 -1.69109414e+00 -1.68690299e+00 -1.68271298e+00
 -1.67852410e+00 -1.67433634e+00 -1.67014972e+00 -1.66596423e+00
 -1.66177986e+00 -1.65759662e+00 -1.65341450e+00 -1.64923350e+00
 -1.64505362e+00 -1.64087486e+00 -1.63669722e+00 -1.63252069e+00
 -1.62834528e+00 -1.62417098e+00 -1.61999779e+00 -1.61582572e+00
 -1.61165475e+00 -1.60748489e+00 -1

¡Linaje de datos guardado en ../data/data_lineage.json!

¡Listo! Archivos COMPLETOS guardados:
Train escalado: ../data/dataset_train_escalado.csv
Test escalado:  ../data/dataset_test_escalado.csv


In [24]:
import sys


src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))

# 2. Agrega 'src' al path de Python (si no está ya)
if src_path not in sys.path:
    sys.path.append(src_path)
    print(f"Carpeta '{src_path}' agregada al path.")

from tools import checksum

Carpeta 'c:\Users\angim\OneDrive\Angi Cosas\Universidad\Laboratorio de datos II\TPF\src' agregada al path.


In [ ]:
checksum(df_train_escalado_final, "df_train")  
checksum(df_test_escalado_final, "df_test")

'Checksum calculado y guardado: 37387ee1ea2ebff9803aa80924233f28'